# First Spark Streaming Example
_____

# Twitter Example
Set up the credentials for a twitter app at https://apps.twitter.com/
    
install python-twitter, a python library to connect your Python to the twitter dev account.

Begin by running the TweetRead.py file. Make sure to add your own IP Adress and your credential keys.

In [1]:
import findspark

In [2]:
# your path will likely not have 'matthew' in it. Change it to reflect your path.
findspark.init('/Users/kevinblum/Apache-Spark/spark-3.1.2-bin-hadoop3.2')

In [3]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.ml import PipelineModel
from pyspark.ml.feature import CountVectorizer

In [4]:
spark = spark = SparkSession \
    .builder \
    .appName("Twitter Streaming App") \
    .getOrCreate()

tweet_df = spark \
    .readStream \
    .format("socket") \
    .option("host", "127.0.0.1") \
    .option("port", 5556) \
    .load()

tweet_df_string = tweet_df.selectExpr("CAST(value AS STRING)")

In [5]:
tweet_df_string = tweet_df_string.withColumnRenamed("value" , "tweet")

In [6]:
writeTweet = tweet_df_string.writeStream.format("csv").\
    option("format" , "append").\
    option("path", "/Users/kevinblum/BigDataProj/SparkTwitterStream/tweets.csv").\
    option("checkpointLocation" , "checkpoints").\
    queryName("tweetquery"). \
    start()


In [7]:
import time
time.sleep(60) 
writeTweet.stop()

In [8]:
import pandas as pd

In [9]:
import os
import glob
import pandas as pd
os.chdir("/Users/kevinblum/BigDataProj/SparkTwitterStream/tweets.csv")

In [10]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]


In [11]:
dataframe = pd.DataFrame()
for i in all_filenames:
    if (os.stat(f"/Users/kevinblum/BigDataProj/SparkTwitterStream/tweets.csv/{i}").st_size != 0):
        try:
            d = pd.read_csv(f"/Users/kevinblum/BigDataProj/SparkTwitterStream/tweets.csv/{i}" , engine="python")
            dt = d.T
            dt = dt.reset_index()
            dataframe = dataframe.append(dt)
        except:
            pass




In [12]:
dataframe = dataframe.rename(columns={"index" : "Tweet"})
dataframe = dataframe.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
dataframe = dataframe.reset_index(drop=True)

In [13]:

dataframe = dataframe[dataframe['Tweet'] != ""]
dataframe = dataframe[dataframe['Tweet'] != "."]
dataframe = dataframe[dataframe['Tweet'] != None]

dataframe = dataframe.reset_index(drop=True)


In [14]:
range1 = range(100, 100 + dataframe['Tweet'].size  )
list1 = list(range1)


In [15]:
dfg = pd.DataFrame(list1 , columns = ['tweet_id'])

In [16]:
dfg = dfg['tweet_id'].astype('str')

In [17]:
list_polarity =[]
for b in range(dataframe['Tweet'].size):
    list_polarity.append(0)
    

In [18]:
dfp = pd.DataFrame(list_polarity , columns = ['polarity'])

In [19]:
dataframe = dataframe.join(dfg)

In [20]:
dataframe = dataframe.join(dfp)

In [21]:
dataframe

,Tweet,tweet_id,polarity
0,2. Lacks the authority for a blan https://t.co...,100,0
1,Imagine if we ran the next primary how we did ...,101,0
2,I also find it amusing that Sen. Manchin indic...,102,0
3,https://t.co/rCinuuXytFRT @danielsgoldman: Whe...,103,0
4,I also find it amusing that Sen. Manchin indic...,104,0
...,...,...,...
101,https://t.co/xgPKLZx3yD,201,0
102,Proving a point.RT @BrajmohanMalvi1: @AshramKo...,202,0
103,There are insurrectionist/coup organizers who ...,203,0
104,I also find it amusing that Sen. Manchin indic...,204,0


In [22]:
sparkDF = spark.createDataFrame(dataframe)

In [23]:
sparkDF.show(5)

+--------------------+--------+--------+
|               Tweet|tweet_id|polarity|
+--------------------+--------+--------+
|2. Lacks the auth...|     100|       0|
|Imagine if we ran...|     101|       0|
|I also find it am...|     102|       0|
|https://t.co/rCin...|     103|       0|
|I also find it am...|     104|       0|
+--------------------+--------+--------+
only showing top 5 rows



In [ ]:
sparkDF.write.option("header",True).csv("/Users/kevinblum/BigDataProj/sparkDF")

In [24]:
import nltk

In [25]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [26]:
from nltk.corpus.reader.wordnet import *
from pyspark.sql.functions import udf
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import word_tokenize
from pyspark.sql.types import StringType,DoubleType,IntegerType
import pyspark.sql.functions as F
wn = nltk.WordNetLemmatizer()
worddict = set(nltk.corpus.words.words())


def preprocessing(text):
    wordset_n = set(wn.lemmatize(w, NOUN) for w in word_tokenize(text.lower().strip()))
    wordset_v = set(wn.lemmatize(w, VERB) for w in wordset_n)
    wordset = set(wn.lemmatize(w, ADJ) for w in wordset_v)
    wordset = wordset & worddict
    return ' '.join(list(wordset))


brand_udf=udf(preprocessing,StringType())
sparkDF=sparkDF.withColumn('text',brand_udf(sparkDF['Tweet']))


In [27]:
from pyspark.ml.feature import  Tokenizer

In [28]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [29]:
sparkDF=tokenizer.transform(sparkDF)

In [30]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")

In [31]:
model1=count.fit(sparkDF)

In [32]:
sparkDF=model1.transform(sparkDF)

In [33]:
from pyspark.ml.feature import  IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [34]:
idfModel = idf.fit(sparkDF)
sparkDF = idfModel.transform(sparkDF)

In [35]:

sparkDF=sparkDF.repartition(10)

In [36]:
sparkDF.printSchema()

root
 |-- Tweet: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- polarity: long (nullable = true)
 |-- text: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)



In [ ]:
sparkDF = sparkDF.withColumn('polarity', sparkDF['polarity'].cast(IntegerType()))

In [41]:
sparkDF.show(10)

+--------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+
|               Tweet|tweet_id|polarity|                text|               words|         rawFeatures|            features|
+--------------------+--------+--------+--------------------+--------------------+--------------------+--------------------+
|Imagine if we ran...|     101|       0|we how leader rig...|[we, how, leader,...|(722,[0,1,2,3,4,5...|(722,[0,1,2,3,4,5...|
|May God help us a...|     115|       0|past not the brot...|[past, not, the, ...|(722,[0,1,2,3,7,1...|(722,[0,1,2,3,7,1...|
|https://t.co/tqaA...|     134|       0|respond accord th...|[respond, accord,...|(722,[0,1,3,4,5,6...|(722,[0,1,3,4,5,6...|
|El pueblo uRT @jo...|     128|       0|el pueblo forecas...|[el, pueblo, fore...|(722,[6,8,13,17,2...|(722,[6,8,13,17,2...|
|Proving a point.T...|     141|       0|we how that for w...|[we, how, that, f...|(722,[0,1,2,3,4,5...|(722,[0,1,2,3,4,5...|


In [37]:
from pyspark.ml.classification import RandomForestClassificationModel
rf_model = RandomForestClassificationModel.load("/Users/kevinblum/BigDataProj/SparkTwitterStream/models/rf_classifier")

In [ ]:
sparkDF

In [38]:
rf_predictions=rf_model.transform(sparkDF.select(['text','words','rawFeatures','features','polarity']))

In [39]:
rf_predictions.select(['prediction']).show(10)

Py4JJavaError: An error occurred while calling o285.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 117.0 failed 1 times, most recent failure: Lost task 0.0 in stage 117.0 (TID 886) (kevins-mbp.home executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ProbabilisticClassificationModel$$Lambda$4486/486288209: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage3.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:345)
	at org.apache.spark.rdd.RDD.$anonfun$mapPar

In [ ]:
import pyspark.sql.functions as F

print(F.length("polarity"))

In [ ]:
sparkDF

In [ ]:
rd_predict = rf_predictions.withColumn("prediction" , rf_predictions["prediction"].cast(StringType()))